In [2]:
# example of a dcgan on cifar10
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from tensorflow.keras.datasets.cifar10 import load_data
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Dropout
from matplotlib import pyplot

In [3]:
# define the standalone discriminator model
def define_discriminator(in_shape=(32,32,3)):
	model = Sequential()
	# normal
	model.add(Conv2D(64, (3,3), padding='same', input_shape=in_shape))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(256, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# classifier
	model.add(Flatten())
	model.add(Dropout(0.4))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

# define the standalone generator model
def define_generator(latent_dim):
	model = Sequential()
	# foundation for 4x4 image
	n_nodes = 256 * 4 * 4
	model.add(Dense(n_nodes, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((4, 4, 256)))
	# upsample to 8x8
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 16x16
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 32x32
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# output layer
	model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
	return model

In [4]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(g_model)
	# add the discriminator
	model.add(d_model)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

In [5]:
# load and prepare cifar10 training images
def load_real_samples():
	# load cifar10 dataset
	(trainX, _), (_, _) = load_data()
	# convert from unsigned ints to floats
	X = trainX.astype('float32')
	# scale from [0,255] to [-1,1]
	X = (X - 127.5) / 127.5
	return X

In [6]:
# select real samples
def generate_real_samples(dataset, n_samples):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# retrieve selected images
	X = dataset[ix]
	# generate 'real' class labels (1)
	y = ones((n_samples, 1))
	return X, y

In [7]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

In [8]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = g_model.predict(x_input)
	# create 'fake' class labels (0)
	y = zeros((n_samples, 1))
	return X, y

In [9]:
# create and save a plot of generated images
def save_plot(examples, epoch, n=7):
	# scale from [-1,1] to [0,1]
	examples = (examples + 1) / 2.0
	# plot images
	for i in range(n * n):
		# define subplot
		pyplot.subplot(n, n, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(examples[i])
	# save plot to file
	filename = 'generated_plot_e%03d.png' % (epoch+1)
	pyplot.savefig(filename)
	pyplot.close()

In [10]:
# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=150):
	# prepare real samples
	X_real, y_real = generate_real_samples(dataset, n_samples)
	# evaluate discriminator on real examples
	_, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
	# prepare fake examples
	x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
	# evaluate discriminator on fake examples
	_, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
	# summarize discriminator performance
	print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
	# save plot
	save_plot(x_fake, epoch)
	# save the generator model tile file
	filename = 'generator_model_%03d.h5' % (epoch+1)
	g_model.save(filename)

In [11]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=200, n_batch=128):
	bat_per_epo = int(dataset.shape[0] / n_batch)
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_epochs):
		# enumerate batches over the training set
		for j in range(bat_per_epo):
			# get randomly selected 'real' samples
			X_real, y_real = generate_real_samples(dataset, half_batch)
			# update discriminator model weights
			d_loss1, _ = d_model.train_on_batch(X_real, y_real)
			# generate 'fake' examples
			X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			# update discriminator model weights
			d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
			# prepare points in latent space as input for the generator
			X_gan = generate_latent_points(latent_dim, n_batch)
			# create inverted labels for the fake samples
			y_gan = ones((n_batch, 1))
			# update the generator via the discriminator's error
			g_loss = gan_model.train_on_batch(X_gan, y_gan)
			# summarize loss on this batch
			print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
				(i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
		# evaluate the model performance, sometimes
		if (i+1) % 10 == 0:
			summarize_performance(i, g_model, d_model, dataset, latent_dim)

In [13]:
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

>1, 1/390, d1=0.688, d2=0.696 g=0.691
>1, 2/390, d1=0.642, d2=0.698 g=0.689
>1, 3/390, d1=0.584, d2=0.704 g=0.683
>1, 4/390, d1=0.513, d2=0.718 g=0.670
>1, 5/390, d1=0.407, d2=0.750 g=0.646
>1, 6/390, d1=0.300, d2=0.808 g=0.612
>1, 7/390, d1=0.201, d2=0.878 g=0.595
>1, 8/390, d1=0.154, d2=0.873 g=0.625
>1, 9/390, d1=0.154, d2=0.801 g=0.704
>1, 10/390, d1=0.144, d2=0.676 g=0.812
>1, 11/390, d1=0.128, d2=0.593 g=0.907
>1, 12/390, d1=0.146, d2=0.557 g=0.945
>1, 13/390, d1=0.154, d2=0.565 g=0.916
>1, 14/390, d1=0.189, d2=0.604 g=0.850
>1, 15/390, d1=0.140, d2=0.647 g=0.809
>1, 16/390, d1=0.134, d2=0.688 g=0.787
>1, 17/390, d1=0.049, d2=0.728 g=0.782
>1, 18/390, d1=0.062, d2=0.799 g=0.776
>1, 19/390, d1=0.111, d2=0.939 g=0.779
>1, 20/390, d1=0.078, d2=0.903 g=0.889
>1, 21/390, d1=0.151, d2=0.650 g=1.089
>1, 22/390, d1=0.254, d2=0.676 g=0.886
>1, 23/390, d1=0.123, d2=0.926 g=0.788
>1, 24/390, d1=0.238, d2=1.119 g=0.684
>1, 25/390, d1=0.167, d2=1.115 g=0.726
>1, 26/390, d1=0.260, d2=0.920 g=0

>3, 73/390, d1=0.669, d2=0.403 g=1.348
>3, 74/390, d1=0.569, d2=0.371 g=1.526
>3, 75/390, d1=0.620, d2=0.392 g=1.440
>3, 76/390, d1=0.622, d2=0.371 g=1.495
>3, 77/390, d1=0.661, d2=0.432 g=1.405
>3, 78/390, d1=0.573, d2=0.427 g=1.567
>3, 79/390, d1=0.558, d2=0.466 g=1.627
>3, 80/390, d1=0.737, d2=0.341 g=1.603
>3, 81/390, d1=0.509, d2=0.351 g=1.622
>3, 82/390, d1=0.527, d2=0.344 g=1.585
>3, 83/390, d1=0.589, d2=0.404 g=1.574
>3, 84/390, d1=0.454, d2=0.363 g=1.602
>3, 85/390, d1=0.611, d2=0.396 g=1.582
>3, 86/390, d1=0.448, d2=0.380 g=1.537
>3, 87/390, d1=0.577, d2=0.435 g=1.570
>3, 88/390, d1=0.510, d2=0.427 g=1.730
>3, 89/390, d1=0.518, d2=0.347 g=1.780
>3, 90/390, d1=0.574, d2=0.406 g=1.829
>3, 91/390, d1=0.425, d2=0.353 g=1.838
>3, 92/390, d1=0.556, d2=0.359 g=1.742
>3, 93/390, d1=0.462, d2=0.376 g=1.869
>3, 94/390, d1=0.499, d2=0.433 g=2.023
>3, 95/390, d1=0.633, d2=0.438 g=1.817
>3, 96/390, d1=0.481, d2=0.482 g=1.932
>3, 97/390, d1=0.515, d2=0.439 g=1.962
>3, 98/390, d1=0.654, d2=

>3, 279/390, d1=0.420, d2=0.492 g=1.327
>3, 280/390, d1=0.499, d2=0.486 g=1.313
>3, 281/390, d1=0.573, d2=0.541 g=1.346
>3, 282/390, d1=0.648, d2=0.482 g=1.350
>3, 283/390, d1=0.538, d2=0.518 g=1.396
>3, 284/390, d1=0.579, d2=0.511 g=1.379
>3, 285/390, d1=0.494, d2=0.502 g=1.314
>3, 286/390, d1=0.622, d2=0.482 g=1.303
>3, 287/390, d1=0.510, d2=0.578 g=1.292
>3, 288/390, d1=0.580, d2=0.557 g=1.221
>3, 289/390, d1=0.508, d2=0.595 g=1.246
>3, 290/390, d1=0.482, d2=0.582 g=1.243
>3, 291/390, d1=0.683, d2=0.536 g=1.112
>3, 292/390, d1=0.587, d2=0.604 g=1.086
>3, 293/390, d1=0.611, d2=0.645 g=1.043
>3, 294/390, d1=0.596, d2=0.609 g=1.125
>3, 295/390, d1=0.695, d2=0.526 g=1.148
>3, 296/390, d1=0.574, d2=0.596 g=1.151
>3, 297/390, d1=0.600, d2=0.549 g=1.154
>3, 298/390, d1=0.526, d2=0.568 g=1.130
>3, 299/390, d1=0.582, d2=0.685 g=1.231
>3, 300/390, d1=0.603, d2=0.488 g=1.213
>3, 301/390, d1=0.609, d2=0.548 g=1.162
>3, 302/390, d1=0.575, d2=0.482 g=1.154
>3, 303/390, d1=0.563, d2=0.529 g=1.175


>4, 98/390, d1=0.524, d2=0.440 g=1.792
>4, 99/390, d1=0.549, d2=0.531 g=1.930
>4, 100/390, d1=0.648, d2=0.434 g=1.941
>4, 101/390, d1=0.693, d2=0.528 g=1.617
>4, 102/390, d1=0.582, d2=0.525 g=1.756
>4, 103/390, d1=0.664, d2=0.468 g=1.937
>4, 104/390, d1=0.816, d2=0.539 g=1.326
>4, 105/390, d1=0.488, d2=0.621 g=1.630
>4, 106/390, d1=0.612, d2=0.415 g=1.615
>4, 107/390, d1=0.642, d2=0.406 g=1.443
>4, 108/390, d1=0.689, d2=0.637 g=1.370
>4, 109/390, d1=0.660, d2=0.533 g=1.502
>4, 110/390, d1=0.598, d2=0.402 g=1.678
>4, 111/390, d1=0.662, d2=0.524 g=1.515
>4, 112/390, d1=0.718, d2=0.493 g=1.443
>4, 113/390, d1=0.566, d2=0.561 g=1.581
>4, 114/390, d1=0.515, d2=0.326 g=1.700
>4, 115/390, d1=0.547, d2=0.326 g=1.592
>4, 116/390, d1=0.556, d2=0.392 g=1.574
>4, 117/390, d1=0.502, d2=0.514 g=1.645
>4, 118/390, d1=0.571, d2=0.365 g=1.592
>4, 119/390, d1=0.499, d2=0.407 g=1.552
>4, 120/390, d1=0.456, d2=0.398 g=1.529
>4, 121/390, d1=0.591, d2=0.426 g=1.522
>4, 122/390, d1=0.509, d2=0.425 g=1.493
>4

>4, 304/390, d1=0.379, d2=0.587 g=1.399
>4, 305/390, d1=0.418, d2=0.606 g=1.194
>4, 306/390, d1=0.458, d2=0.816 g=1.081
>4, 307/390, d1=0.486, d2=0.921 g=1.104
>4, 308/390, d1=0.547, d2=0.761 g=1.128
>4, 309/390, d1=0.530, d2=0.645 g=1.284
>4, 310/390, d1=0.465, d2=0.719 g=1.308
>4, 311/390, d1=0.636, d2=0.606 g=1.407
>4, 312/390, d1=0.614, d2=0.628 g=1.375
>4, 313/390, d1=0.614, d2=0.608 g=1.398
>4, 314/390, d1=0.648, d2=0.711 g=1.503
>4, 315/390, d1=0.696, d2=0.586 g=1.366
>4, 316/390, d1=0.797, d2=0.544 g=1.359
>4, 317/390, d1=0.713, d2=0.485 g=1.186
>4, 318/390, d1=0.656, d2=0.644 g=1.110
>4, 319/390, d1=0.706, d2=0.697 g=1.183
>4, 320/390, d1=0.584, d2=0.645 g=1.248
>4, 321/390, d1=0.692, d2=0.541 g=1.144
>4, 322/390, d1=0.685, d2=0.567 g=1.098
>4, 323/390, d1=0.672, d2=0.530 g=1.083
>4, 324/390, d1=0.637, d2=0.556 g=1.107
>4, 325/390, d1=0.585, d2=0.562 g=1.100
>4, 326/390, d1=0.560, d2=0.552 g=1.115
>4, 327/390, d1=0.592, d2=0.529 g=1.168
>4, 328/390, d1=0.598, d2=0.519 g=1.232


...

>90, 243/390, d1=0.699, d2=0.589 g=1.037
>90, 244/390, d1=0.588, d2=0.550 g=0.983
>90, 245/390, d1=0.661, d2=0.619 g=1.077
>90, 246/390, d1=0.729, d2=0.583 g=1.013
>90, 247/390, d1=0.726, d2=0.568 g=1.122
>90, 248/390, d1=0.639, d2=0.541 g=1.055
>90, 249/390, d1=0.684, d2=0.530 g=1.040
>90, 250/390, d1=0.674, d2=0.590 g=0.992
>90, 251/390, d1=0.571, d2=0.543 g=0.943
>90, 252/390, d1=0.578, d2=0.650 g=1.033
>90, 253/390, d1=0.642, d2=0.631 g=1.039
>90, 254/390, d1=0.682, d2=0.629 g=0.974
>90, 255/390, d1=0.566, d2=0.599 g=0.946
>90, 256/390, d1=0.657, d2=0.631 g=0.929
>90, 257/390, d1=0.650, d2=0.610 g=0.898
>90, 258/390, d1=0.595, d2=0.651 g=0.938
>90, 259/390, d1=0.610, d2=0.610 g=0.974
>90, 260/390, d1=0.625, d2=0.618 g=1.005
>90, 261/390, d1=0.584, d2=0.621 g=1.035
>90, 262/390, d1=0.549, d2=0.517 g=1.025
>90, 263/390, d1=0.729, d2=0.550 g=0.957
>90, 264/390, d1=0.621, d2=0.626 g=0.970
>90, 265/390, d1=0.582, d2=0.635 g=0.957
>90, 266/390, d1=0.594, d2=0.621 g=0.960
>90, 267/390, d1

>91, 55/390, d1=0.572, d2=0.590 g=0.999
>91, 56/390, d1=0.672, d2=0.606 g=1.005
>91, 57/390, d1=0.639, d2=0.634 g=0.992
>91, 58/390, d1=0.626, d2=0.575 g=1.004
>91, 59/390, d1=0.628, d2=0.541 g=0.931
>91, 60/390, d1=0.696, d2=0.601 g=0.923
>91, 61/390, d1=0.582, d2=0.603 g=0.922
>91, 62/390, d1=0.642, d2=0.565 g=0.963
>91, 63/390, d1=0.598, d2=0.654 g=0.911
>91, 64/390, d1=0.549, d2=0.702 g=0.931
>91, 65/390, d1=0.585, d2=0.692 g=0.957
>91, 66/390, d1=0.656, d2=0.581 g=1.012
>91, 67/390, d1=0.589, d2=0.690 g=0.954
>91, 68/390, d1=0.677, d2=0.595 g=1.016
>91, 69/390, d1=0.615, d2=0.689 g=1.033
>91, 70/390, d1=0.673, d2=0.549 g=0.984
>91, 71/390, d1=0.573, d2=0.651 g=1.025
>91, 72/390, d1=0.641, d2=0.570 g=1.020
>91, 73/390, d1=0.688, d2=0.623 g=0.989
>91, 74/390, d1=0.581, d2=0.501 g=1.016
>91, 75/390, d1=0.686, d2=0.603 g=1.039
>91, 76/390, d1=0.601, d2=0.612 g=1.095
>91, 77/390, d1=0.598, d2=0.527 g=1.030
>91, 78/390, d1=0.682, d2=0.616 g=0.978
>91, 79/390, d1=0.653, d2=0.638 g=1.021


>91, 257/390, d1=0.579, d2=0.729 g=0.946
>91, 258/390, d1=0.607, d2=0.574 g=1.046
>91, 259/390, d1=0.705, d2=0.540 g=1.015
>91, 260/390, d1=0.751, d2=0.583 g=1.020
>91, 261/390, d1=0.650, d2=0.525 g=1.039
>91, 262/390, d1=0.801, d2=0.607 g=1.022
>91, 263/390, d1=0.631, d2=0.589 g=1.101
>91, 264/390, d1=0.627, d2=0.571 g=0.988
>91, 265/390, d1=0.677, d2=0.597 g=0.948
>91, 266/390, d1=0.610, d2=0.638 g=1.005
>91, 267/390, d1=0.614, d2=0.626 g=0.952
>91, 268/390, d1=0.589, d2=0.649 g=0.932
>91, 269/390, d1=0.541, d2=0.539 g=1.010
>91, 270/390, d1=0.696, d2=0.627 g=1.020
>91, 271/390, d1=0.618, d2=0.584 g=0.978
>91, 272/390, d1=0.638, d2=0.593 g=1.006
>91, 273/390, d1=0.617, d2=0.595 g=0.995
>91, 274/390, d1=0.701, d2=0.627 g=0.908
>91, 275/390, d1=0.598, d2=0.586 g=0.917
>91, 276/390, d1=0.637, d2=0.592 g=0.952
>91, 277/390, d1=0.600, d2=0.551 g=0.959
>91, 278/390, d1=0.506, d2=0.602 g=1.007
>91, 279/390, d1=0.541, d2=0.574 g=0.994
>91, 280/390, d1=0.567, d2=0.580 g=1.016
>91, 281/390, d1

>92, 69/390, d1=0.647, d2=0.555 g=1.038
>92, 70/390, d1=0.671, d2=0.701 g=1.078
>92, 71/390, d1=0.672, d2=0.506 g=1.074
>92, 72/390, d1=0.671, d2=0.548 g=1.087
>92, 73/390, d1=0.710, d2=0.626 g=0.988
>92, 74/390, d1=0.555, d2=0.671 g=0.934
>92, 75/390, d1=0.666, d2=0.653 g=0.966
>92, 76/390, d1=0.633, d2=0.593 g=0.980
>92, 77/390, d1=0.612, d2=0.615 g=1.041
>92, 78/390, d1=0.601, d2=0.586 g=1.016
>92, 79/390, d1=0.632, d2=0.613 g=0.983
>92, 80/390, d1=0.660, d2=0.552 g=1.039
>92, 81/390, d1=0.616, d2=0.553 g=1.009
>92, 82/390, d1=0.629, d2=0.627 g=1.004
>92, 83/390, d1=0.596, d2=0.559 g=0.998
>92, 84/390, d1=0.611, d2=0.631 g=0.989
>92, 85/390, d1=0.596, d2=0.486 g=0.935
>92, 86/390, d1=0.534, d2=0.638 g=1.080
>92, 87/390, d1=0.594, d2=0.518 g=1.034
>92, 88/390, d1=0.542, d2=0.493 g=0.990
>92, 89/390, d1=0.688, d2=0.619 g=0.885
>92, 90/390, d1=0.565, d2=0.619 g=0.975
>92, 91/390, d1=0.612, d2=0.657 g=0.973
>92, 92/390, d1=0.593, d2=0.680 g=0.930
>92, 93/390, d1=0.615, d2=0.586 g=0.998


KeyboardInterrupt: 